1. Download checkpoints from 
https://drive.google.com/drive/folders/16IXv-w6xpHhEQiSNjRSI8S5wS4QDjGKE?usp=sharing

2. The checkpoints for T1 and T2 are seperate

3. We use ukbiobank preprocessed images. Link to download ukbiobank preprocessing documentation and code is 
https://git.fmrib.ox.ac.uk/falmagro/UK_biobank_pipeline_v_1
https://biobank.ctsu.ox.ac.uk/crystal/crystal/docs/brain_mri.pdf


4. The only custom imports are model_128 and dataset

5. Packages needed are listed in requirments if imports fail

In [2]:
import torch
from glob import glob
from monai import transforms
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt

import torch
from torch.nn import functional as F
from torch import nn
from model_128 import *
from dataset import *

# from dataset import *
from tqdm import tqdm
import numpy as np
import nibabel
import matplotlib.pyplot as plt

In [3]:
"List of names of MRI"
T1_files= list()

In [ ]:
model = model.to(device)
checkpoint = torch.load(T1_ckpt, map_location=device)
model = model.load_state_dict(checkpoint["state_dict"])

In [ ]:
T1_ds = aedataset(
    datafile=T1_files, modality= "T1_unbiased_linear"
)
val_dataloader = torch.utils.data.DataLoader(
    T1_ds, batch_size=16, pin_memory=True, num_workers=32, shuffle=False
)

In [ ]:
loss_fn = torch.nn.MSELoss(
             reduction="none"
        )

In [ ]:
model = model.eval()

In [ ]:
losses_val = []
with torch.no_grad():
    for data in val_dataloader:
        img, mask = data
        img = img.to(device)
        mask = mask.to(device)
        recon, lin1 = model(img)
        loss = loss_fn(img, recon)
        loss1 = loss.squeeze(1) * mask
        loss2 = loss1.sum()
        loss3 = loss2 / mask.sum()
        losses_val.append(loss3.cpu().numpy())
        


In [ ]:
print("Mean validation loss")
print(np.mean(losses_val))